In [1]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "BTC"
to_date = '2021-06-21'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']
    return date




In [2]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])

In [3]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [4]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [5]:
drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,14,0,38330000.0,38394000.0,38292000.0,72.863689
1,13,30,38300000.0,38400000.0,38123000.0,206.220691
2,13,0,38123000.0,38461000.0,38106000.0,183.268314
3,12,30,38303000.0,38326000.0,37911000.0,200.707883
4,12,0,37913000.0,38390000.0,37911000.0,255.477421


In [6]:
ups, downs = df['tradePrice'].loc[::-1].diff() ,df['tradePrice'].loc[::-1].diff()
ups[ups<0] =0
downs[downs>0] = 0
period = 14 
au = ups.abs().ewm(com = period-1, min_periods = period).mean() 
ad = downs.abs().ewm(com = period-1, min_periods = period).mean() 
RS = au/ad 
RSI = pd.Series(100 - (100/(1+RS))) 
RSI = RSI.loc[::-1]
print(RSI)
df['RSI'] = RSI

0      63.130709
1      62.769201
2      60.655704
3      64.091391
4      59.472963
         ...    
312          NaN
313          NaN
314          NaN
315          NaN
316          NaN
Name: tradePrice, Length: 317, dtype: float64


In [7]:
# CCI 오예
N = 20
D = 0
m = 0
for i in range(3):
    D = 0
    m = 0
    sm= 0

    M = ((df['tradePrice'][i] + df['lowPrice'][i] + df['highPrice'][i]) / 3)

    for j in range(N):
        jM = (df['tradePrice'][i+j] + df['lowPrice'][i+j] + df['highPrice'][i+j]) / 3
        sm +=jM
    sm = (sm/N)

    for k in range(N):
        kM = (df['tradePrice'][i+k] + df['lowPrice'][i+k] + df['highPrice'][i+k]) / 3
        D += abs(sm-kM)
    D = D/N
    print('D : {}  // M : {} // m : {}'.format(D,M,sm))

    CCI = (M - sm) / (D * 0.015)

    print(CCI)


D : 576720.0000000007  // M : 38338666.666666664 // m : 37535133.33333333
92.88543641435878
D : 564716.6666666672  // M : 38274333.333333336 // m : 37466783.333333336
95.33394327538876
D : 546850.0  // M : 38230000.0 // m : 37402883.333333336
100.83406987494001


In [8]:
s_data, l_data = df['tradePrice'].loc[::-1],df['tradePrice'].loc[::-1]
s_data
l_data

316    40830000.0
315    40838000.0
314    40938000.0
313    40870000.0
312    41017000.0
          ...    
4      37913000.0
3      38303000.0
2      38123000.0
1      38300000.0
0      38330000.0
Name: tradePrice, Length: 317, dtype: float64

In [9]:
# MACD
period = 12
s_data, l_data = df['tradePrice'].loc[::-1],df['tradePrice'].loc[::-1]
# s_data, l_data = df['tradePrice'].diff() ,df['tradePrice'].diff()
a12 = s_data.abs().ewm(com = period-1, min_periods = period).mean() 
a12.head(15)


316             NaN
315             NaN
314             NaN
313             NaN
312             NaN
311             NaN
310             NaN
309             NaN
308             NaN
307             NaN
306             NaN
305    4.138548e+07
304    4.144915e+07
303    4.148463e+07
302    4.150365e+07
Name: tradePrice, dtype: float64

In [10]:
s_data = df.loc[::-1]
s_data

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume,RSI
316,00,0,40830000.0,40980000.0,40600000.0,345.947578,NaN
315,00,30,40838000.0,40960000.0,40755000.0,149.095600,NaN
314,01,0,40938000.0,40950000.0,40700000.0,163.617955,NaN
313,01,30,40870000.0,40938000.0,40759000.0,71.849834,NaN
312,02,0,41017000.0,41040000.0,40788000.0,84.941353,NaN
...,...,...,...,...,...,...,...
4,12,0,37913000.0,38390000.0,37911000.0,255.477421,59.472963
3,12,30,38303000.0,38326000.0,37911000.0,200.707883,64.091391
2,13,0,38123000.0,38461000.0,38106000.0,183.268314,60.655704
1,13,30,38300000.0,38400000.0,38123000.0,206.220691,62.769201


In [11]:
def fnMACD(m_Df, m_NumFast=12, m_NumSlow=26, m_NumSignal=9):
    m_Df['EMAFast'] = m_Df['tradePrice'].ewm( span = m_NumFast, min_periods = m_NumFast - 1).mean()
    m_Df['EMASlow'] = m_Df['tradePrice'].ewm( span = m_NumSlow, min_periods = m_NumSlow - 1).mean()
    m_Df['MACD'] = m_Df['EMAFast'] - m_Df['EMASlow']
    m_Df['MACDSignal'] = m_Df['MACD'].ewm( span = m_NumSignal, min_periods = m_NumSignal-1).mean()
    m_Df['MACDDiff'] = m_Df['MACD'] - m_Df['MACDSignal']
    return m_Df

s_data = fnMACD(s_data)
s_data

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume,RSI,EMAFast,EMASlow,MACD,MACDSignal,MACDDiff
316,00,0,40830000.0,40980000.0,40600000.0,345.947578,NaN,NaN,NaN,NaN,NaN,NaN
315,00,30,40838000.0,40960000.0,40755000.0,149.095600,NaN,NaN,NaN,NaN,NaN,NaN
314,01,0,40938000.0,40950000.0,40700000.0,163.617955,NaN,NaN,NaN,NaN,NaN,NaN
313,01,30,40870000.0,40938000.0,40759000.0,71.849834,NaN,NaN,NaN,NaN,NaN,NaN
312,02,0,41017000.0,41040000.0,40788000.0,84.941353,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4,12,0,37913000.0,38390000.0,37911000.0,255.477421,59.472963,3.770919e+07,3.735131e+07,357872.838616,239462.242304,118410.596312
3,12,30,38303000.0,38326000.0,37911000.0,200.707883,64.091391,3.780054e+07,3.742181e+07,378733.484767,267316.490796,111416.993971
2,13,0,38123000.0,38461000.0,38106000.0,183.268314,60.655704,3.785015e+07,3.747375e+07,376402.270386,289133.646714,87268.623672
1,13,30,38300000.0,38400000.0,38123000.0,206.220691,62.769201,3.791936e+07,3.753495e+07,384405.993930,308188.116157,76217.877772


In [12]:
s_data.drop(['EMAFast','EMASlow'],axis=1)
s_data

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume,RSI,EMAFast,EMASlow,MACD,MACDSignal,MACDDiff
316,00,0,40830000.0,40980000.0,40600000.0,345.947578,NaN,NaN,NaN,NaN,NaN,NaN
315,00,30,40838000.0,40960000.0,40755000.0,149.095600,NaN,NaN,NaN,NaN,NaN,NaN
314,01,0,40938000.0,40950000.0,40700000.0,163.617955,NaN,NaN,NaN,NaN,NaN,NaN
313,01,30,40870000.0,40938000.0,40759000.0,71.849834,NaN,NaN,NaN,NaN,NaN,NaN
312,02,0,41017000.0,41040000.0,40788000.0,84.941353,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4,12,0,37913000.0,38390000.0,37911000.0,255.477421,59.472963,3.770919e+07,3.735131e+07,357872.838616,239462.242304,118410.596312
3,12,30,38303000.0,38326000.0,37911000.0,200.707883,64.091391,3.780054e+07,3.742181e+07,378733.484767,267316.490796,111416.993971
2,13,0,38123000.0,38461000.0,38106000.0,183.268314,60.655704,3.785015e+07,3.747375e+07,376402.270386,289133.646714,87268.623672
1,13,30,38300000.0,38400000.0,38123000.0,206.220691,62.769201,3.791936e+07,3.753495e+07,384405.993930,308188.116157,76217.877772


In [18]:
s_data = s_data.loc[::-1]
s_data.tail()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume,RSI,EMAFast,EMASlow,MACD,MACDSignal,MACDDiff
4,12,0,37913000.0,38390000.0,37911000.0,255.477421,59.472963,3.770919e+07,3.735131e+07,357872.838616,239462.242304,118410.596312
3,12,30,38303000.0,38326000.0,37911000.0,200.707883,64.091391,3.780054e+07,3.742181e+07,378733.484767,267316.490796,111416.993971
2,13,0,38123000.0,38461000.0,38106000.0,183.268314,60.655704,3.785015e+07,3.747375e+07,376402.270386,289133.646714,87268.623672
1,13,30,38300000.0,38400000.0,38123000.0,206.220691,62.769201,3.791936e+07,3.753495e+07,384405.993930,308188.116157,76217.877772
0,14,0,38330000.0,38394000.0,38292000.0,72.863689,63.130709,3.798253e+07,3.759385e+07,388689.184809,324288.329888,64400.854921
